# Solutions to the machine-Learning tutorial for classification

## Objective
1. Predict diagnostic label from functional connectome
2. Predict MRI acquisition site from functional connectome

## Dataset
[ABIDE (Autism)](https://nilearn.github.io/modules/generated/nilearn.datasets.fetch_abide_pcp.html)

## Preprocessing
1. Generate region-to-region connectivity matrix (i.e. connectome) from functional timeseries data
    - ROIs are defined based on [harvard_oxford atlas](https://nilearn.github.io/modules/generated/nilearn.datasets.fetch_atlas_harvard_oxford.html) or [AAL atlas](https://nilearn.github.io/modules/generated/nilearn.datasets.fetch_atlas_aal.html)
2. Apply dimensionality reduction (Optional)

## Model 
1. Logistic regression
2. Random Forest

## Cross-validation
1. k-fold
2. shuffle-split

## post-hoc analysis
Compare model performance for predicting Dx labels vs. MRI acquisition site


# Let's begin! 
### First we import some useful python libraries...

In [ ]:
## Imports
from nilearn import datasets
from nilearn.connectome import ConnectivityMeasure
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Download data

In [ ]:
n_subjects = 1000
parcel = 'rois_ho' # 'rois_ho' or 'rois_aal
data = datasets.fetch_abide_pcp(n_subjects=n_subjects,derivatives=[parcel],data_dir='./') 

## Phenotypes and Demographics

In [ ]:
pheno = pd.DataFrame(data['phenotypic']).drop(columns=['i','Unnamed_0'])
pheno.head()

In [ ]:
site_counts = pheno['SITE_ID'].value_counts()
dx_counts = pheno['DX_GROUP'].value_counts()

print(f'Dx count:\n{dx_counts}\n\nScanning site_counts:\n{site_counts}')

## MRI features

### These are stored in a list, where each list element is a subject-specific feature matrix
### Subject specific feature matrix: timepoints x ROIs
### ROIs are defined based on [harvard_oxford atlas](https://nilearn.github.io/modules/generated/nilearn.datasets.fetch_atlas_harvard_oxford.html) or [AAL atlas](https://nilearn.github.io/modules/generated/nilearn.datasets.fetch_atlas_aal.html)

In [ ]:
features = data[parcel]

print(f'Number of samples: {len(features)}')

subject_feature_shape = features[0].shape

print(f'subject_feature_shape: {subject_feature_shape}')

## Let's see how the atlas looks like

In [ ]:
from nilearn import plotting
if parcel == 'rois_ho':
    atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
else:
    atlas = datasets.fetch_atlas_aal()

plotting.plot_roi(atlas.maps, draw_cross=False, title=parcel)

## And the subject-specific feature matrix

In [ ]:
f, ax = plt.subplots(figsize=(15,10))
g = sns.heatmap(features[0].T, ax=ax)
g.set_xlabel('timepoint')
g.set_ylabel('ROI')
g.set_title('Functional data timeseries')


## Preprocessing / feature engineering

### Commonly functional (timeseries) neuroimaging data is represented as functional connectome aka network ake graph. 

In [ ]:
connectome_matrix = ConnectivityMeasure(kind='correlation')
connectome_matrix = connectome_matrix.fit_transform([features[0]])[0]
print(f'Shape connectome: {connectome_matrix.shape}')

f, ax = plt.subplots(figsize=(15,10))
g = sns.heatmap(connectome_matrix, ax=ax)
g.set_xlabel('ROI')
g.set_ylabel('ROI')
g.set_title('Connectome')


In [ ]:
# Extract lower (or upper) triangle entrees (excluding diagonal)
tril_idx = np.tril_indices(len(connectome_matrix),k=1) 
features_flat = connectome_matrix[tril_idx]
print(f'Number of features per subject: {len(features_flat)}')

## Now do this for all subjects!

In [ ]:
# defs are definitely useful! 

def extract_connectome_features(func_data, measure):
    ''' A function to calculate connnectome based on timeseries data and similarity measure
    '''
    connectome_matrix = measure.fit_transform([func_data])[0]
    tril_idx = np.tril_indices(len(connectome_matrix),k=1) 
    flat_features = connectome_matrix[tril_idx]

    return flat_features


## Note: here we are pre-processing each image independently i.e. not using any group-level information for scaling / normalization / feature transformation (e.g. PCA). Therefore there is no "double dipping" or leakage of information from a test set. This sort of independent image pre-processing, we can do on entire dataset without creating train-test splits first and then defining feature-set on the training data only. 

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

flat_features_list = []
for func_data in features:
    flat_features = extract_connectome_features(func_data, correlation_measure)
    flat_features_list.append(flat_features)

print(f'Length of flat_features_list {len(flat_features_list)}')

## Input data matrix (n_samples, n_features)

In [ ]:
X = np.array(flat_features_list)

print(f'Input data (X) shape: {X.shape}')

# Memory intensive #

# g = sns.heatmap(flat_features_array)
# g.set_xlabel('Connection strength')
# g.set_ylabel('ROI')
# g.set_title('Connectome')

## Output labels (y): Diagnosis


In [ ]:
from sklearn import preprocessing

outcome = 'DX_GROUP'
y = pheno[outcome]
y_counts = y.value_counts()

print(f'Unique output clasess:\n{y_counts}')

# Encode labels to integer categories
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

## Okay now we have our input data (X) and output data (y) in the following format
<img src="./QLS_ML_terminology.png" alt="terms" width="800"/>


## Create train-test split
- 80/20 ratio
- Stratify 

In [ ]:
from sklearn.model_selection import train_test_split

test_subset_fraction = 0.2 # 
stratification = y 

X_train, X_test, y_train, y_test = train_test_split(
                                                    X, # input features
                                                    y, # output labels
                                                    test_size = test_subset_fraction, 
                                                    shuffle = True, # shuffle dataset
                                                                    # before splitting
                                                    stratify = stratification, 
                                                    random_state = 123 # same shuffle each time
                                                    )

# print the size of our training and test groups
print('training:', len(X_train), 'testing:', len(X_test))

## Okay finally, let's train your first model!

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model = 'LR' # 'LR' or 'RF'

if model == 'RF':
    clf = RandomForestClassifier(max_depth=3, class_weight='balanced', random_state=0)
elif model == 'LR':
    clf = LogisticRegression(penalty='l1', C=1, class_weight='balanced', solver='saga', random_state=0)
else:
    print(f'Unknown model: {model}')

print(f'Using model: {model}')

clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train)
print(f'train acc: {train_acc:.3f}')


## Evaluate on test set
- accuracy
- confusion_matrix
- precision_recall_fscore 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)

test_acc = clf.score(X_test, y_test)
print(f'test acc: {test_acc:.3f}')

test_cm = confusion_matrix(y_test, y_pred)

### Note the difference between train and test accuracy. If the difference is large, we are most likely overfitting the model to the train set. 

#### Things to try:
- Increase regularization
- Reduce dimensionality of X

In [ ]:
f, ax = plt.subplots(figsize=(10,5))
g = sns.heatmap(test_cm, annot=True, ax=ax)
g.set_title('Confusion matrix')

In [ ]:
p,r,f1,_ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(f'model: {model}, outcome: {outcome}\n Acc:{test_acc:.2f}, precision: {p:.2f}, recall: {r:.2f}, f1: {f1:.2f}')


## Now let's predict scanning site!

In [ ]:
outcome = 'SITE_ID'
y = pheno[outcome]
y_counts = y.value_counts()

print(f'Unique output clasess:\n{y_counts}')

# Encode labels to integer categories
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

## Create train-test split
- 80/20 ratio
- Stratify 

In [ ]:
from sklearn.model_selection import train_test_split

test_subset_fraction = 0.2 # 
stratification = y 

X_train, X_test, y_train, y_test = train_test_split(
                                                    X, # input features
                                                    y, # output labels
                                                    test_size = test_subset_fraction, 
                                                    shuffle = True, # shuffle dataset
                                                                    # before splitting
                                                    stratify = stratification, 
                                                    random_state = 123 # same shuffle each time
                                                    )

# print the size of our training and test groups
print('training:', len(X_train), 'testing:', len(X_test))

## Fit the model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model = 'LR' # 'LR' or 'RF' 

if model == 'RF':
    clf = RandomForestClassifier(max_depth=3, class_weight='balanced', random_state=0)
elif model == 'LR':
    clf = LogisticRegression(penalty='l2', C=1, class_weight='balanced', solver='saga', random_state=0)
else:
    print(f'Unknown model: {model}')

print(f'Using model: {model}')

clf.fit(X_train, y_train)

train_acc = clf.score(X_train, y_train)
print(f'train acc: {train_acc:.3f}')


## Evaluate on test set
- accuracy
- confusion_matrix
- precision_recall_fscore 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)

test_acc = clf.score(X_test, y_test)
print(f'test acc: {test_acc:.3f}')

test_cm = confusion_matrix(y_test, y_pred)

In [ ]:
f, ax = plt.subplots(figsize=(15,10))
g = sns.heatmap(test_cm, annot=True, ax=ax)
g.set_title('Confusion matrix')

In [ ]:
p,r,f1,_ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

print(f'model: {model}, outcome: {outcome}\n Acc:{test_acc:.2f}, precision: {p:.2f}, recall: {r:.2f}, f1: {f1:.2f}')